In [1]:
!cp -r ../input/d/romanofrancesco/recsys-repo/RecSys_Course_AT_PoliMi-master/* ./

In [2]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_path = "../input/recommender-system-2021-challenge-polimi/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                header=0)
URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]

userID_unique = URM_all_dataframe["UserID"].unique()
itemID_unique = URM_all_dataframe["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)

In [3]:
import numpy as np
import scipy.sparse as sps
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
import time, sys
from tqdm import tqdm
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# os.environ["PYTHONWARNINGS"] = ('ignore::exceptions.ConvergenceWarning:sklearn.linear_model')
# os.environ["PYTHONWARNINGS"] = ('ignore:Objective did not converge:ConvergenceWarning:')

class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available
    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    @ignore_warnings(category=ConvergenceWarning)
    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100,**earlystopping_kwargs):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)

        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]

            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1

            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)

In [4]:
recommender_SLIMElasticNet = SLIMElasticNetRecommender(URM_all)
recommender_SLIMElasticNet.fit(l1_ratio = 0.001404017101088145, alpha = 0.06305313484275951, positive_only = True, topK = 2594)

SLIMElasticNetRecommender: Processed 1123 ( 6.2%) in 5.00 min. Items per second: 3.74
SLIMElasticNetRecommender: Processed 2235 (12.4%) in 10.00 min. Items per second: 3.72
SLIMElasticNetRecommender: Processed 3376 (18.7%) in 15.00 min. Items per second: 3.75
SLIMElasticNetRecommender: Processed 4493 (24.9%) in 20.01 min. Items per second: 3.74
SLIMElasticNetRecommender: Processed 5633 (31.2%) in 25.01 min. Items per second: 3.75
SLIMElasticNetRecommender: Processed 6755 (37.4%) in 30.01 min. Items per second: 3.75
SLIMElasticNetRecommender: Processed 7880 (43.6%) in 35.01 min. Items per second: 3.75
SLIMElasticNetRecommender: Processed 8999 (49.8%) in 40.01 min. Items per second: 3.75
SLIMElasticNetRecommender: Processed 10179 (56.4%) in 45.02 min. Items per second: 3.77
SLIMElasticNetRecommender: Processed 11318 (62.7%) in 50.02 min. Items per second: 3.77
SLIMElasticNetRecommender: Processed 12483 (69.1%) in 55.02 min. Items per second: 3.78
SLIMElasticNetRecommender: Processed 1360

In [5]:
import pyximport
pyximport.install()
!python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1822,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:620:
/opt/conda/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In function ‘__

In [6]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
recommender_SLIM_BPR_Cython = SLIM_BPR_Cython(URM_all)
recommender_SLIM_BPR_Cython.fit(epochs=990, sgd_mode = "sgd", topK = 515, lambda_i = 1e-05, 
                lambda_j = 0.02698741041205141, learning_rate = 0.002605133534059859)

SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 16761.00 MB (93.11%) of 18001.00 MB, required is 1304.51 MB. Using dense matrix.
Processed 13650 (100.0%) in 1.51 sec. BPR loss is 3.31E-01. Sample per second: 9030
SLIM_BPR_Recommender: Epoch 1 of 990. Elapsed time 0.91 sec
Processed 13650 (100.0%) in 1.36 sec. BPR loss is 1.27E+00. Sample per second: 10034
SLIM_BPR_Recommender: Epoch 2 of 990. Elapsed time 1.76 sec
Processed 13650 (100.0%) in 1.16 sec. BPR loss is 2.21E+00. Sample per second: 11728
SLIM_BPR_Recommender: Epoch 3 of 990. Elapsed time 2.56 sec
Processed 13650 (100.0%) in 0.96 sec. BPR loss is 3.05E+00. Sample per second: 14241
SLIM_BPR_Recommender: Epoch 4 of 990. Elapsed time 3.35 sec
Processed 13650 (100.0%) in 1.76 sec. BPR loss is 3.86E+00. Sample per second: 7768
SLIM_BPR_Recommender: Epoch 5 of 990. Elapsed time 4.15 sec
Processed 13650 (100.0%) in 1.56 sec. BPR loss is 4.63E+00. Sample per second: 8767
SLIM_BPR_Recommender: Epoch 6 

In [7]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

recommender_mf_ials = IALSRecommender(URM_all)
recommender_mf_ials.fit(num_factors = 49, confidence_scaling = "linear", alpha = 0.7374080457174151, epsilon = 2.4848274666036083, reg = 4.473996989962275e-05, epochs = 75.0)

IALSRecommender: Epoch 1 of 75.0. Elapsed time 24.76 sec
IALSRecommender: Epoch 2 of 75.0. Elapsed time 42.28 sec
IALSRecommender: Epoch 3 of 75.0. Elapsed time 1.03 min
IALSRecommender: Epoch 4 of 75.0. Elapsed time 1.33 min
IALSRecommender: Epoch 5 of 75.0. Elapsed time 1.66 min
IALSRecommender: Epoch 6 of 75.0. Elapsed time 1.94 min
IALSRecommender: Epoch 7 of 75.0. Elapsed time 2.28 min
IALSRecommender: Epoch 8 of 75.0. Elapsed time 2.58 min
IALSRecommender: Epoch 9 of 75.0. Elapsed time 2.90 min
IALSRecommender: Epoch 10 of 75.0. Elapsed time 3.21 min
IALSRecommender: Epoch 11 of 75.0. Elapsed time 3.52 min
IALSRecommender: Epoch 12 of 75.0. Elapsed time 3.83 min
IALSRecommender: Epoch 13 of 75.0. Elapsed time 4.13 min
IALSRecommender: Epoch 14 of 75.0. Elapsed time 4.44 min
IALSRecommender: Epoch 15 of 75.0. Elapsed time 4.76 min
IALSRecommender: Epoch 16 of 75.0. Elapsed time 5.06 min
IALSRecommender: Epoch 17 of 75.0. Elapsed time 5.40 min
IALSRecommender: Epoch 18 of 75.0. Ela

In [8]:
from numpy import linalg as LA
from Recommenders.BaseRecommender import BaseRecommender

class DifferentLossScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of three predictions scores
    R = R1*alpha + R2*beta + R3*(1-alpha-beta)
    
    Class from Dacrema exercise modified by Antonio Ercolani
    The original took as input 2 recommender

    """

    RECOMMENDER_NAME = "DifferentLossScoresHybridRecommender"


    def __init__(self, URM_train, recommender_1, recommender_2, recommender_3):
        super(DifferentLossScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        self.recommender_3 = recommender_3
        
        
        
    def fit(self, norm, alpha = 0.5, beta = 0.5):

        self.alpha = alpha
        self.beta = beta
        self.norm = norm


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)
        item_weights_3 = self.recommender_3._compute_item_score(user_id_array)

        norm_item_weights_1 = LA.norm(item_weights_1, self.norm)
        norm_item_weights_2 = LA.norm(item_weights_2, self.norm)
        norm_item_weights_3 = LA.norm(item_weights_3, self.norm)
        
        
        if norm_item_weights_1 == 0:
            raise ValueError("Norm {} of item weights for recommender 1 is zero. Avoiding division by zero".format(self.norm))
        
        if norm_item_weights_2 == 0:
            raise ValueError("Norm {} of item weights for recommender 2 is zero. Avoiding division by zero".format(self.norm))
            
        if norm_item_weights_3 == 0:
            raise ValueError("Norm {} of item weights for recommender 3 is zero. Avoiding division by zero".format(self.norm))
        
        item_weights = item_weights_1 / norm_item_weights_1 * self.alpha + item_weights_2 / norm_item_weights_2 * self.beta + item_weights_3 / norm_item_weights_3 * (1-self.alpha-self.beta)

        return item_weights

In [9]:
recommender = DifferentLossScoresHybridRecommender(URM_all, recommender_SLIMElasticNet, recommender_SLIM_BPR_Cython, recommender_mf_ials)
recommender.fit(2, 0.54, 0.3)

In [10]:
test_users = pd.read_csv('../input/recommender-system-2021-challenge-polimi/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
13645,13645
13646,13646
13647,13647
13648,13648


In [11]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user,cutoff = 10))

In [12]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submission.csv', index=False)